In [1]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
X_train.shape, X_val.shape, X_test.shape

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


((55000, 784), (5000, 784), (10000, 784))

In [2]:
# Pre-processing: normalizing
def normalize(X):
    # max scale for images 255= 2**8= 8 bit grayscale for each channel
    return (X - X.mean(axis=0)) #/ X.std(axis=0)

X_train, X_val, X_test = normalize(X=X_train), normalize(X=X_val), normalize(X=X_test)

In [3]:
# Model
import impl.layer as l # or from impl.layer import *
from impl.loss import * # import all functions from impl.loss file # import impl.loss as loss_func
from sklearn.utils import shuffle as skshuffle

class FFNN:

    def __init__(self, D, C, H, L):
        self.L = L # layers
        self.C = C # classes
        self.losses = {'train':[], 'train_acc':[], 
                       'valid':[], 'valid_acc':[], 
                       'test':[], 'test_acc':[]}
        
        self.model = []
        self.W_fixed = []
        self.grads = []
        self.dy_prev = np.zeros((1, C))
        self.y_prev = np.zeros((1, C))
        low, high = -1, 1
        
        # Input layer: weights/ biases
        m = dict(W=np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.), b=np.zeros((1, H)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.)
        self.W_fixed.append(m)
        # Input layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})

        # Hidden layers: weights/ biases
        m_L = []
        for _ in range(L):
            m = dict(W=np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, H)))
            m_L.append(m)
        self.model.append(m_L)
        # Fixed feedback weight
        m_L = []
        for _ in range(L):
            m = np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.)
            m_L.append(m)
        self.W_fixed.append(m_L)
        # Hidden layer: gradients
        grad_L = []
        for _ in range(L):
            grad_L.append({key: np.zeros_like(val) for key, val in self.model[1][0].items()})
        self.grads.append(grad_L)
        
        # Output layer: weights/ biases
        m = dict(W=np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, C)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.)
        self.W_fixed.append(m)
        # Output layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
        
    def fc_forward(self, X, W, b):
        out = (X @ W) + b
        cache = (W, X)
        return out, cache

    def fc_backward(self, dout, cache, W_fixed):
        W, X = cache

        dW = X.T @ dout
        db = np.sum(dout, axis=0).reshape(1, -1) # db_1xn
        
#         dX = dout @ W.T # vanilla Backprop
        dX = dout @ W_fixed.T # fba backprop

        return dX, dW, db

    def train_forward(self, X, train):
        caches, ys = [], []
        
        # Input layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[0]['W'], b=self.model[0]['b']) # X_1xD, y_1xc
        y, nl_cache = l.tanh_forward(X=y)
#         y, nl_cache = l.sigmoid_forward(X=y) # non-linearity/ activation
#         y -= l.sigmoid(0.0) # zero-centered/ mean
#         y *= 2.0 # uni-var/ std
        if train:
            caches.append((fc_cache, nl_cache))
        X = y.copy() # pass to the next layer
        
        # Hidden layers
        fc_caches, nl_caches = [], []
        for layer in range(self.L):
            y, fc_cache = self.fc_forward(X=X, W=self.model[1][layer]['W'], b=self.model[1][layer]['b'])
            y, nl_cache = l.tanh_forward(X=y)
#             y, nl_cache = l.sigmoid_forward(X=y) # non-linearity/ activation
#             y -= l.sigmoid(0.0) # zero-centered/ mean
#             y *= 2.0 # uni-var/ std
            X = y.copy() # pass to next layer
            if train:
                fc_caches.append(fc_cache)
                nl_caches.append(nl_cache)
        if train:
            caches.append((fc_caches, nl_caches)) # caches[1]            
        
        # Output layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[2]['W'], b=self.model[2]['b'])
        y_prob = l.softmax(X=y)
        if train:
            caches.append(fc_cache)

        return y_prob, caches # for backpropating the error

    def cross_entropy(self, y_prob, y_train):
        m = y_prob.shape[0]

        #         prob = l.softmax(y_pred)
        log_like = -np.log(y_prob[range(m), y_train] + l.eps) # to avoid the devision by zero
        data_loss = np.sum(log_like) / m

        return data_loss

    def dcross_entropy(self, y_prob, y_train): # this is equal for both since the reg_loss (noise) derivative is ZERO.
        m = y_prob.shape[0]

        #         grad_y = l.softmax(y_pred)
        grad_y = y_prob
        grad_y[range(m), y_train] -= 1.
        grad_y /= m

        return grad_y

    def loss_function(self, y_prob, y_train):
        
        loss = self.cross_entropy(y_prob, y_train) # softmax is included
        dy = self.dcross_entropy(y_prob, y_train) # dsoftmax is included

        return loss, dy
        
    def train_backward(self, dy, caches, y):
        grads = self.grads.copy() # initialized by Zero in every iteration/epoch
        dy_prev = self.dy_prev.copy() # for temporal differencing
        self.dy_prev = dy.copy() # next iteration/ epoch
#         y_prev = self.y_prev.copy() # for temporal differencing
#         self.y_prev = y.copy() # next iteration/ epoch
        
        # Output layer
        fc_cache = caches[2]
        # softmax_backward is included in dcross_entropy.
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[2])
        dy = dX.copy()
#         dy =  dy @ self.W_fixed[2].T # done
        dy_prev =  dy_prev @ self.W_fixed[2].T
#         y =  y @ self.W_fixed[2].T # done
#         y_prev =  y_prev @ self.W_fixed[2].T
        grads[2]['W'] = dW
        grads[2]['b'] = db

        # Hidden layer
        fc_caches, nl_caches = caches[1]
        for layer in reversed(range(self.L)):
#             dy = l.tanh_backward(cache=nl_caches[layer], dout=dy) # diffable function
#             dy = l.sigmoid_backward(cache=nl_caches[layer], dout=dy) # diffable function
            dy *= dy - dy_prev # temporal diff instead of differentiable function
#             dy *= y - y_prev # temporal diff instead of differentiable function
            dX, dW, db = self.fc_backward(dout=dy, cache=fc_caches[layer], W_fixed=self.W_fixed[1][layer])
            dy = dX.copy()
#             dy =  dy @ self.W_fixed[2].T # done
            dy_prev =  dy_prev @ self.W_fixed[1][layer].T
#             y =  y @ self.W_fixed[1][layer].T # done
#             y_prev =  y_prev @ self.W_fixed[1][layer].T
            grads[1][layer]['W'] = dW
            grads[1][layer]['b'] = db
        
        # Input layer
        fc_cache, nl_cache = caches[0]
#         dy = l.tanh_backward(cache=nl_cache, dout=dy) # diffable function
#         dy = l.sigmoid_backward(cache=nl_caches[layer], dout=dy) # diffable function
        dy *= dy - dy_prev # temporal diff instead of differentiable function
#         dy *= y - y_prev # temporal diff instead of differentiable function
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[0])
        grads[0]['W'] = dW
        grads[0]['b'] = db

        return dX, grads
    
    def test(self, X):
        y_prob, _ = self.train_forward(X, train=False)
        
        # if self.mode == 'classification':
        y_pred = np.argmax(y_prob, axis=1) # for loss ==err
        
        return y_pred, y_prob
        
    def get_minibatch(self, X, y, minibatch_size, shuffle):
        minibatches = []

        if shuffle:
            X, y = skshuffle(X, y)

        for i in range(0, X.shape[0], minibatch_size):
            X_mini = X[i:i + minibatch_size]
            y_mini = y[i:i + minibatch_size]
            minibatches.append((X_mini, y_mini))

        return minibatches

    def sgd(self, train_set, val_set, alpha, mb_size, n_iter, print_after):
        X_train, y_train = train_set
        X_val, y_val = val_set

        # Epochs
        for iter in range(1, n_iter + 1):

            # Minibatches
            minibatches = self.get_minibatch(X_train, y_train, mb_size, shuffle=True)
            idx = np.random.randint(0, len(minibatches))
            X_mini, y_mini = minibatches[idx]
            
            # Train the model
            y_prob, caches = self.train_forward(X_mini, train=True)
            _, dy = self.loss_function(y_prob, y_mini)
            _, grads = self.train_backward(dy, caches, y_prob)
            
            # Update the model for input layer
            for key in grads[0].keys():
                self.model[0][key] -= alpha * grads[0][key]

            # Update the model for the hidden layers
            for layer in range(self.L):
                for key in grads[1][layer].keys():
                    self.model[1][layer][key] -= alpha * grads[1][layer][key]

            # Update the model for output layer
            for key in grads[2].keys():
                self.model[2][key] -= alpha * grads[2][key]
            
            # Training accuracy
            y_pred, y_prob = self.test(X_mini)
            loss, _ = self.loss_function(y_prob, y_mini) # softmax is included in entropy loss function
            self.losses['train'].append(loss)
            acc = np.mean(y_pred == y_mini) # confusion matrix
            self.losses['train_acc'].append(acc)

            # Validate the updated model
            y_pred, y_prob = self.test(X_val)
            valid_loss, _ = self.loss_function(y_prob, y_val) # softmax is included in entropy loss function
            self.losses['valid'].append(valid_loss)
            valid_acc = np.mean(y_pred == y_val) # confusion matrix
            self.losses['valid_acc'].append(valid_acc)
            
            # Test the final model
            y_pred, y_prob = nn.test(X_test)
            test_loss, _ = self.loss_function(y_prob, y_test) # softmax is included in entropy loss function
            self.losses['test'].append(test_loss)
            test_acc = np.mean(y_pred == y_test)
            self.losses['test_acc'].append(test_acc)
#             print('Test accuracy mean: {:.4f}, std: {:.4f}, loss: {:.4f}'.
#             format(acc.mean(), acc.std(), loss))
            
            # Print the model info: loss & accuracy or err & acc
            if iter % print_after == 0:
                print('Iter-{}, train loss-{:.4f}, acc-{:.4f}, valid loss-{:.4f}, acc-{:.4f}, test loss-{:.4f}, acc-{:.4f}'.format(
                   iter, loss, acc, valid_loss, valid_acc, test_loss, test_acc))

In [ ]:
# Hyper-parameters
n_iter = 100000 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 50 # 2**10==1024 # width, timestep for sequential data or minibatch size
print_after = 10 # n_iter//10 # print loss for train, valid, and test
num_hidden_units = 32 # number of kernels/ filters in each layer
num_input_units = X_train.shape[1] # noise added at the input lavel as input noise we can use dX or for more improvement
num_output_units = y_train.max() + 1 # number of classes in this classification problem
num_layers = 2 # depth 

# Build the model/NN and learn it: running session.
nn = FFNN(C=num_output_units, D=num_input_units, H=num_hidden_units, L=num_layers)

nn.sgd(train_set=(X_train, y_train), val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

Iter-10, train loss-2.3219, acc-0.1200, valid loss-2.3191, acc-0.1034, test loss-2.3200, acc-0.0937
Iter-20, train loss-2.3418, acc-0.1000, valid loss-2.3190, acc-0.1040, test loss-2.3199, acc-0.0940
Iter-30, train loss-2.3174, acc-0.0600, valid loss-2.3188, acc-0.1044, test loss-2.3198, acc-0.0942
Iter-40, train loss-2.3151, acc-0.1800, valid loss-2.3187, acc-0.1046, test loss-2.3196, acc-0.0946
Iter-50, train loss-2.3148, acc-0.0800, valid loss-2.3185, acc-0.1046, test loss-2.3195, acc-0.0950
Iter-60, train loss-2.3034, acc-0.1400, valid loss-2.3184, acc-0.1052, test loss-2.3193, acc-0.0953
Iter-70, train loss-2.3074, acc-0.1200, valid loss-2.3183, acc-0.1052, test loss-2.3192, acc-0.0954
Iter-80, train loss-2.3214, acc-0.0800, valid loss-2.3181, acc-0.1056, test loss-2.3190, acc-0.0955
Iter-90, train loss-2.3169, acc-0.1200, valid loss-2.3180, acc-0.1058, test loss-2.3189, acc-0.0957
Iter-100, train loss-2.3270, acc-0.1200, valid loss-2.3178, acc-0.1058, test loss-2.3188, acc-0.0957

Iter-830, train loss-2.3097, acc-0.1000, valid loss-2.3076, acc-0.1178, test loss-2.3085, acc-0.1113
Iter-840, train loss-2.3053, acc-0.1400, valid loss-2.3075, acc-0.1184, test loss-2.3084, acc-0.1118
Iter-850, train loss-2.3083, acc-0.0600, valid loss-2.3073, acc-0.1184, test loss-2.3083, acc-0.1117
Iter-860, train loss-2.3190, acc-0.1000, valid loss-2.3072, acc-0.1188, test loss-2.3081, acc-0.1119
Iter-870, train loss-2.3070, acc-0.0800, valid loss-2.3071, acc-0.1190, test loss-2.3080, acc-0.1121
Iter-880, train loss-2.3010, acc-0.0800, valid loss-2.3069, acc-0.1188, test loss-2.3078, acc-0.1122
Iter-890, train loss-2.2944, acc-0.1000, valid loss-2.3068, acc-0.1192, test loss-2.3077, acc-0.1122
Iter-900, train loss-2.3009, acc-0.1600, valid loss-2.3066, acc-0.1196, test loss-2.3075, acc-0.1124
Iter-910, train loss-2.3344, acc-0.1000, valid loss-2.3065, acc-0.1202, test loss-2.3074, acc-0.1124
Iter-920, train loss-2.2926, acc-0.1200, valid loss-2.3063, acc-0.1200, test loss-2.3073, a

Iter-1640, train loss-2.3018, acc-0.1200, valid loss-2.2962, acc-0.1404, test loss-2.2972, acc-0.1326
Iter-1650, train loss-2.3215, acc-0.0400, valid loss-2.2961, acc-0.1406, test loss-2.2970, acc-0.1332
Iter-1660, train loss-2.2978, acc-0.1200, valid loss-2.2960, acc-0.1406, test loss-2.2969, acc-0.1330
Iter-1670, train loss-2.2979, acc-0.1400, valid loss-2.2958, acc-0.1408, test loss-2.2967, acc-0.1333
Iter-1680, train loss-2.2788, acc-0.1200, valid loss-2.2957, acc-0.1410, test loss-2.2966, acc-0.1334
Iter-1690, train loss-2.3103, acc-0.0800, valid loss-2.2955, acc-0.1412, test loss-2.2965, acc-0.1335
Iter-1700, train loss-2.2888, acc-0.1000, valid loss-2.2954, acc-0.1416, test loss-2.2963, acc-0.1338
Iter-1710, train loss-2.3112, acc-0.1000, valid loss-2.2953, acc-0.1414, test loss-2.2962, acc-0.1339
Iter-1720, train loss-2.2933, acc-0.1400, valid loss-2.2951, acc-0.1416, test loss-2.2960, acc-0.1346
Iter-1730, train loss-2.3068, acc-0.1000, valid loss-2.2950, acc-0.1422, test loss

Iter-2450, train loss-2.3015, acc-0.1000, valid loss-2.2852, acc-0.1582, test loss-2.2861, acc-0.1556
Iter-2460, train loss-2.2904, acc-0.1600, valid loss-2.2850, acc-0.1586, test loss-2.2860, acc-0.1559
Iter-2470, train loss-2.3017, acc-0.0600, valid loss-2.2849, acc-0.1590, test loss-2.2858, acc-0.1560
Iter-2480, train loss-2.3011, acc-0.1200, valid loss-2.2848, acc-0.1590, test loss-2.2857, acc-0.1565
Iter-2490, train loss-2.2801, acc-0.1600, valid loss-2.2846, acc-0.1594, test loss-2.2856, acc-0.1563
Iter-2500, train loss-2.2655, acc-0.1600, valid loss-2.2845, acc-0.1592, test loss-2.2854, acc-0.1568
Iter-2510, train loss-2.2815, acc-0.1800, valid loss-2.2844, acc-0.1600, test loss-2.2853, acc-0.1574
Iter-2520, train loss-2.2906, acc-0.1400, valid loss-2.2842, acc-0.1602, test loss-2.2852, acc-0.1573
Iter-2530, train loss-2.3126, acc-0.1000, valid loss-2.2841, acc-0.1612, test loss-2.2850, acc-0.1576
Iter-2540, train loss-2.2872, acc-0.2000, valid loss-2.2840, acc-0.1616, test loss

Iter-3260, train loss-2.2675, acc-0.1800, valid loss-2.2744, acc-0.1844, test loss-2.2753, acc-0.1822
Iter-3270, train loss-2.2804, acc-0.1800, valid loss-2.2743, acc-0.1846, test loss-2.2752, acc-0.1823
Iter-3280, train loss-2.2719, acc-0.1600, valid loss-2.2742, acc-0.1846, test loss-2.2750, acc-0.1827
Iter-3290, train loss-2.2569, acc-0.1800, valid loss-2.2740, acc-0.1850, test loss-2.2749, acc-0.1832
Iter-3300, train loss-2.2748, acc-0.1800, valid loss-2.2739, acc-0.1854, test loss-2.2748, acc-0.1836
Iter-3310, train loss-2.2696, acc-0.1600, valid loss-2.2738, acc-0.1852, test loss-2.2747, acc-0.1840
Iter-3320, train loss-2.2671, acc-0.1600, valid loss-2.2737, acc-0.1858, test loss-2.2745, acc-0.1839
Iter-3330, train loss-2.2549, acc-0.2000, valid loss-2.2735, acc-0.1860, test loss-2.2744, acc-0.1844
Iter-3340, train loss-2.2644, acc-0.2200, valid loss-2.2734, acc-0.1862, test loss-2.2742, acc-0.1848
Iter-3350, train loss-2.2793, acc-0.1800, valid loss-2.2733, acc-0.1864, test loss

...

Iter-57740, train loss-1.7546, acc-0.6000, valid loss-1.8191, acc-0.4954, test loss-1.8200, acc-0.4907
Iter-57750, train loss-1.8657, acc-0.3400, valid loss-1.8190, acc-0.4956, test loss-1.8199, acc-0.4907
Iter-57760, train loss-1.8238, acc-0.5600, valid loss-1.8190, acc-0.4956, test loss-1.8199, acc-0.4907
Iter-57770, train loss-1.7442, acc-0.5000, valid loss-1.8189, acc-0.4958, test loss-1.8198, acc-0.4907
Iter-57780, train loss-1.8949, acc-0.4600, valid loss-1.8189, acc-0.4958, test loss-1.8198, acc-0.4908
Iter-57790, train loss-1.8370, acc-0.5000, valid loss-1.8188, acc-0.4960, test loss-1.8197, acc-0.4906
Iter-57800, train loss-1.9071, acc-0.4600, valid loss-1.8188, acc-0.4964, test loss-1.8197, acc-0.4908
Iter-57810, train loss-1.8447, acc-0.4800, valid loss-1.8187, acc-0.4964, test loss-1.8196, acc-0.4908
Iter-57820, train loss-1.7387, acc-0.5800, valid loss-1.8187, acc-0.4968, test loss-1.8196, acc-0.4907
Iter-57830, train loss-1.7473, acc-0.5200, valid loss-1.8186, acc-0.4964,

Iter-58540, train loss-1.7658, acc-0.5400, valid loss-1.8150, acc-0.4972, test loss-1.8159, acc-0.4915
Iter-58550, train loss-1.8862, acc-0.4000, valid loss-1.8149, acc-0.4974, test loss-1.8159, acc-0.4914
Iter-58560, train loss-1.8126, acc-0.4400, valid loss-1.8149, acc-0.4972, test loss-1.8158, acc-0.4914
Iter-58570, train loss-1.7531, acc-0.5000, valid loss-1.8148, acc-0.4972, test loss-1.8158, acc-0.4914
Iter-58580, train loss-1.8659, acc-0.5200, valid loss-1.8148, acc-0.4972, test loss-1.8157, acc-0.4915
Iter-58590, train loss-1.8516, acc-0.5200, valid loss-1.8147, acc-0.4974, test loss-1.8157, acc-0.4915
Iter-58600, train loss-1.7925, acc-0.5600, valid loss-1.8147, acc-0.4972, test loss-1.8156, acc-0.4916
Iter-58610, train loss-1.7680, acc-0.5800, valid loss-1.8146, acc-0.4974, test loss-1.8156, acc-0.4916
Iter-58620, train loss-1.7842, acc-0.5000, valid loss-1.8146, acc-0.4972, test loss-1.8155, acc-0.4915
Iter-58630, train loss-1.8324, acc-0.4600, valid loss-1.8145, acc-0.4972,

Iter-59340, train loss-1.7987, acc-0.4200, valid loss-1.8110, acc-0.4986, test loss-1.8119, acc-0.4928
Iter-59350, train loss-1.7445, acc-0.6400, valid loss-1.8109, acc-0.4982, test loss-1.8118, acc-0.4929
Iter-59360, train loss-1.9643, acc-0.4400, valid loss-1.8109, acc-0.4982, test loss-1.8118, acc-0.4929
Iter-59370, train loss-1.8991, acc-0.4200, valid loss-1.8108, acc-0.4982, test loss-1.8118, acc-0.4928
Iter-59380, train loss-1.7397, acc-0.5800, valid loss-1.8108, acc-0.4982, test loss-1.8117, acc-0.4928
Iter-59390, train loss-1.8799, acc-0.4200, valid loss-1.8107, acc-0.4982, test loss-1.8117, acc-0.4928
Iter-59400, train loss-1.8287, acc-0.5200, valid loss-1.8107, acc-0.4982, test loss-1.8116, acc-0.4927
Iter-59410, train loss-1.8844, acc-0.5000, valid loss-1.8106, acc-0.4984, test loss-1.8116, acc-0.4928
Iter-59420, train loss-1.8362, acc-0.4600, valid loss-1.8106, acc-0.4984, test loss-1.8115, acc-0.4927
Iter-59430, train loss-1.8473, acc-0.5200, valid loss-1.8105, acc-0.4982,

Iter-60140, train loss-1.8266, acc-0.5000, valid loss-1.8070, acc-0.4990, test loss-1.8079, acc-0.4930
Iter-60150, train loss-1.8505, acc-0.4800, valid loss-1.8070, acc-0.4992, test loss-1.8079, acc-0.4929
Iter-60160, train loss-1.7538, acc-0.6000, valid loss-1.8069, acc-0.4996, test loss-1.8078, acc-0.4931
Iter-60170, train loss-1.9024, acc-0.4800, valid loss-1.8069, acc-0.4992, test loss-1.8078, acc-0.4931
Iter-60180, train loss-1.7978, acc-0.5400, valid loss-1.8068, acc-0.4994, test loss-1.8077, acc-0.4930
Iter-60190, train loss-1.8476, acc-0.4600, valid loss-1.8068, acc-0.4994, test loss-1.8077, acc-0.4929
Iter-60200, train loss-1.7611, acc-0.5200, valid loss-1.8067, acc-0.4994, test loss-1.8076, acc-0.4929
Iter-60210, train loss-1.7770, acc-0.6000, valid loss-1.8067, acc-0.4994, test loss-1.8076, acc-0.4930
Iter-60220, train loss-1.8962, acc-0.4200, valid loss-1.8066, acc-0.4994, test loss-1.8076, acc-0.4928
Iter-60230, train loss-1.8860, acc-0.4200, valid loss-1.8066, acc-0.4994,

Iter-60940, train loss-1.8190, acc-0.5600, valid loss-1.8031, acc-0.5000, test loss-1.8040, acc-0.4947
Iter-60950, train loss-1.8646, acc-0.4200, valid loss-1.8031, acc-0.5000, test loss-1.8040, acc-0.4947
Iter-60960, train loss-1.8338, acc-0.4400, valid loss-1.8030, acc-0.5000, test loss-1.8039, acc-0.4947
Iter-60970, train loss-1.8673, acc-0.3600, valid loss-1.8030, acc-0.4996, test loss-1.8039, acc-0.4948
Iter-60980, train loss-1.8441, acc-0.4400, valid loss-1.8030, acc-0.5000, test loss-1.8038, acc-0.4947
Iter-60990, train loss-1.8682, acc-0.4800, valid loss-1.8029, acc-0.4998, test loss-1.8038, acc-0.4948
Iter-61000, train loss-1.7980, acc-0.5200, valid loss-1.8029, acc-0.5000, test loss-1.8037, acc-0.4947
Iter-61010, train loss-1.7365, acc-0.6400, valid loss-1.8028, acc-0.4998, test loss-1.8037, acc-0.4947
Iter-61020, train loss-1.8626, acc-0.5200, valid loss-1.8028, acc-0.4996, test loss-1.8036, acc-0.4948
Iter-61030, train loss-1.9121, acc-0.4200, valid loss-1.8027, acc-0.4996,

In [ ]:
# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(nn.losses['train'], label='Train loss')
plt.plot(nn.losses['valid'], label='Valid loss')
plt.plot(nn.losses['test'], label='Test loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(nn.losses['train_acc'], label='Train accuracy')
plt.plot(nn.losses['valid_acc'], label='Valid accuracy')
plt.plot(nn.losses['test_acc'], label='Test accuracy')
plt.legend()
plt.show()